In [2]:
input_file = open("untitled.txt", 'r')

bwt = input_file.readline().replace('\n', '')
patterns = input_file.readline().replace('\n', '').split()

def get_pattern_count_beter_bw(bwt, patterns) :
    symbols = set(bwt)
    current_count = {ch:0 for ch in symbols}
    count = {0:{ch:current_count[ch] for ch in symbols}}
    for i in xrange(len(bwt)):
        current_count[bwt[i]] += 1
        count[i+1] = {ch:current_count[ch] for ch in symbols}
    sorted_bwt = sorted(bwt)
    first_occurrence = {ch:sorted_bwt.index(ch) for ch in set(bwt)}

    return [better_bw_matching(bwt, first_occurrence, count, pattern) for pattern in patterns]


def better_bw_matching(bwt, first_occurrence, count, pattern):
    top, bottom = 0, len(bwt) - 1
    while top <= bottom:
        if pattern != '':
            symbol = pattern[-1]
            pattern = pattern[:-1]
            if symbol in bwt[top:bottom+1]:
                top = first_occurrence[symbol] + count[top][symbol]
                bottom = first_occurrence[symbol] + count[bottom+1][symbol] - 1
            else:
                return 0
        else:
            return bottom - top + 1

pattern_count = map(str, get_pattern_count_beter_bw(bwt, patterns))
print(' '.join(pattern_count))

1 0 0 1 1 1 0 0 0 0 0 1 1 0 1 1 1 1 0 0 1 1 0 0 0 0 1 1 1 1 0 1 1 1 1 1 0 1 0 0 0 0 0 0 0 0 0 0 1 0 1 0 1 1 0 1 0 1 0 0 1 1 1 1 0 0 0 0 1 1 0 1 0 0 1 1 1 0 0 0 0 1 1 1 0 0 0 1 0 1 0 1 1 1 1 1 0 0 0 0 1 1 1 1 1 0 1 0 1 1 1 0 1 0 0 1 1 1 0 1 1 1 0 0 1 0 1 1 0 1 0 1 0 0 0 0 0 0 0 1 1 0 1 0 0 1 0 1 1 0 1 0 1 1 1 0 0 1 1 0 1 1 1 0 0 0 1 0 1 0 1 1 1 0 0 1 0 1 1 0 1 1 1 0 0 1 1 0 0 0 1 1 0 1 1 1 0 0 0 1 1 0 1 1 1 0 1 0 1 1 0 0 1 1 0 1 1 1 1 1 1 0 0 1 1 0 0 0 0 1 0 0 0 1 0 0 0 1 0 1 0 1 1 0 0 0 0 0 1 0 0 0 0 1 0 0 0 1 0 1 0 0 1 1 1 0 1 1 0 1 0 1 0 1 1 1 0 1 1 0 1 0 0 0 1 1 0 0 1 0 0 1 1 0 1 1 0 0 0 0 1 1 0 1 0 0 1 1 0 0 1 1 0 0 0 1 1 1 1 0 1 0 0 1 1 1 0 0 1 1 1 1 0 1 0 1 1 1 0 0 1 0 1 1 0 0 0 1 1 1 1 0 1 1 0 0 1 1 1 1 1 0 0 0 1 1 0 0 1 1 0 0 1 0 0 0 0 0 1 0 0 0 0 0 0 0 0 1 0 0 0 1 0 0 0 0 1 1 1 0 1 1 1 1 1 0 1 0 0 1 1 0 0 0 1 1 1 0 1 1 0 0 1 1 1 0 0 1 0 0 0 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 0 1 1 1 0 0 1 0 0 0 1 0 1 1 0 0 1 0 0 0 0 0 1 0 1 0 1 1 1 0 0 1 0 0 1 1 1 0 1 1 1 1 0 1 1 1 0 1 0 1 1 0 1 1 

In [5]:
input_file = open("untitled.txt", 'r')

word = input_file.readline().replace('\n', '')
patterns = input_file.read().splitlines()

def burrows_wheeler_transform(word) :
    word += ['', '$'][word[-1] != '$']
    L = len(word)
    cyclic_rot_index = lambda i, n: word[(n-i) % L]
    cyclic_comp = lambda i, j, n=0: [1, -1][cyclic_rot_index(i,n) < cyclic_rot_index(j,n)] if cyclic_rot_index(i,n) != cyclic_rot_index(j,n) else cyclic_comp(i,j,n+1)
    cyclic_sort = sorted(xrange(len(word)), cmp=cyclic_comp)
    return ''.join([cyclic_rot_index(i, -1) for i in cyclic_sort])

def construct_suffix_array(word) :
    word += ['', '$'][word[-1] != '$']
    suffix_comp = lambda i,j: [1, -1][word[i] < word[j]] if word[i] != word[j] else suffix_comp(i+1,j+1)
    suffix_array = sorted(xrange(len(word)), cmp=suffix_comp)
    return suffix_array

def get_multi_pattern_count(word, patterns) :
    bwt = burrows_wheeler_transform(word)
    suffix_array = construct_suffix_array(word)
    symbols = set(bwt)
    current_count = {ch:0 for ch in symbols}
    count = {0:{ch:current_count[ch] for ch in symbols}}
    for i in xrange(len(bwt)):
        current_count[bwt[i]] += 1
        count[i+1] = {ch:current_count[ch] for ch in symbols}
    sorted_bwt = sorted(bwt)
    first_occurrence = {ch:sorted_bwt.index(ch) for ch in set(bwt)}
    matches = []
    for pattern in patterns : matches += multi_pattern_match_bw(bwt, suffix_array, first_occurrence, count, pattern)
    return matches

def multi_pattern_match_bw(bwt, suffix_array, first_occurrence, count, pattern) :
    top, bottom = 0, len(bwt) - 1
    while top <= bottom :
        if pattern != '' :
            symbol = pattern[-1]
            pattern = pattern[:-1]
            if symbol in bwt[top:bottom+1] :
                top = first_occurrence[symbol] + count[top][symbol]
                bottom = first_occurrence[symbol] + count[bottom+1][symbol] - 1
            else : return []
        else : return [suffix_array[i] for i in xrange(top, bottom + 1)]

pattern_locations = get_multi_pattern_count(word, patterns)
pattern_locations = map(str, sorted(pattern_locations))
print ' '.join(pattern_locations)

9 20 21 31 35 36 45 46 47 48 49 50 51 52 60 61 62 63 74 83 84 104 105 119 120 121 130 135 139 140 141 142 143 144 145 146 166 167 168 173 182 183 185 186 187 188 189 195 203 204 207 208 209 219 228 229 231 232 233 234 248 257 258 259 261 262 263 268 273 274 275 276 277 286 294 303 304 319 320 321 322 323 324 325 326 327 344 345 346 347 348 349 350 351 364 370 371 377 388 389 393 398 408 417 419 428 434 435 438 439 440 441 442 443 444 445 446 447 450 451 462 463 476 477 478 479 480 481 482 483 499 500 501 502 513 514 515 516 517 518 519 520 521 534 535 536 537 542 543 544 551 552 553 571 575 594 595 596 597 598 599 600 601 617 620 629 630 640 641 642 643 649 650 651 652 653 654 663 664 665 666 667 668 669 670 678 679 680 681 682 683 684 685 696 697 705 729 730 731 732 733 734 735 736 737 760 772 780 781 782 783 791 792 793 794 797 798 799 800 801 802 803 804 807 819 824 825 828 829 830 831 839 840 846 847 848 851 859 860 861 883 896 897 898 899 900 901 902 903 907 908 909 917 921 922 93